# Connect to ImSwitch Embedded Kernel

This notebook demonstrates how to connect to the embedded Jupyter kernel running inside ImSwitch.

## Prerequisites

1. Start ImSwitch with the embedded kernel:
   ```bash
   python -m imswitch --with-kernel --headless
   ```

2. The embedded kernel should show a message like:
   ```
   To connect another client to this kernel, use:
       --existing kernel-XXXXX.json
   ```

## Method 1: Use Jupyter Console (Recommended)

The easiest way is to use Jupyter console in a separate terminal:

```bash
jupyter console --existing
```

This will automatically connect to the most recent kernel.

## Method 2: Connect from this notebook

Run the cells below to find and connect to the embedded kernel:

In [1]:
# Find the embedded kernel connection file
import os
import glob
import json
from datetime import datetime

def find_embedded_kernel():
    """Find the most recent ImSwitch embedded kernel connection file"""
    try:
        import jupyter_core.paths
        runtime_dir = jupyter_core.paths.jupyter_runtime_dir()
    except ImportError:
        runtime_dir = os.path.expanduser('~/.local/share/jupyter/runtime')
    
    if not os.path.exists(runtime_dir):
        return None
    
    # Find all kernel files
    kernel_files = glob.glob(os.path.join(runtime_dir, 'kernel-*.json'))
    
    if not kernel_files:
        return None
    
    # Return the most recently modified one
    latest_file = max(kernel_files, key=os.path.getmtime)
    
    # Check if it's recent (within last hour)
    mod_time = os.path.getmtime(latest_file)
    if (datetime.now().timestamp() - mod_time) > 3600:  # 1 hour
        print(f"Warning: Latest kernel file is old ({datetime.fromtimestamp(mod_time)})")
    
    return latest_file

connection_file = find_embedded_kernel()
if connection_file:
    print(f"Found kernel connection file: {connection_file}")
    
    # Read connection info
    with open(connection_file, 'r') as f:
        connection_info = json.load(f)
    
    print(f"Kernel details:")
    print(f"  IP: {connection_info.get('ip', 'unknown')}")
    print(f"  Shell port: {connection_info.get('shell_port', 'unknown')}")
    print(f"  Created: {datetime.fromtimestamp(os.path.getmtime(connection_file))}")
    
else:
    print("No embedded kernel found!")
    print("Make sure ImSwitch is running with --with-kernel flag")

Found kernel connection file: /Users/bene/Library/Jupyter/runtime/kernel-365f086e-3dbb-40fa-985c-0d58ed9d47bb.json
Kernel details:
  IP: 127.0.0.1
  Shell port: 65009
  Created: 2025-09-08 23:18:48.772357


In [2]:
# Connect to the embedded kernel using a subprocess
if connection_file:
    print(f"To connect to the embedded kernel, run this in a terminal:")
    print(f"jupyter console --existing {os.path.basename(connection_file)}")
    print()
    print("Once connected, you'll have access to:")
    print("- detectorsManager")
    print("- lasersManager")
    print("- stageManager")
    print("- master_controller")
    print("- app")
    print("- mainView")
    print("- config")
    print()
    print("Example commands to try:")
    print(">>> detectorsManager.snap_image()")
    print(">>> lasersManager.getAllDeviceNames()")
    print(">>> stageManager.move_to(x=100, y=50)")

To connect to the embedded kernel, run this in a terminal:
jupyter console --existing kernel-365f086e-3dbb-40fa-985c-0d58ed9d47bb.json

Once connected, you'll have access to:
- detectorsManager
- lasersManager
- stageManager
- master_controller
- app
- mainView
- config

Example commands to try:
>>> detectorsManager.snap_image()
>>> lasersManager.getAllDeviceNames()
>>> stageManager.move_to(x=100, y=50)


## Alternative: Create a new kernel client

For advanced users, you can create a client that connects to the embedded kernel programmatically:

In [3]:
# Advanced: Connect programmatically (requires jupyter_client)
try:
    from jupyter_client import BlockingKernelClient
    import time
    
    if connection_file and os.path.exists(connection_file):
        print("Creating kernel client...")
        
        # Create a client
        client = BlockingKernelClient()
        client.load_connection_file(connection_file)
        client.start_channels()
        
        # Test the connection
        msg_id = client.execute('print("Hello from embedded kernel!")')
        
        # Get the response
        try:
            reply = client.get_shell_msg(timeout=5)
            print(f"Kernel response: {reply['content']['status']}")
            
            # Get any output
            while True:
                try:
                    msg = client.get_iopub_msg(timeout=1)
                    if msg['msg_type'] == 'stream':
                        print(f"Output: {msg['content']['text'].strip()}")
                    elif msg['msg_type'] == 'execute_result':
                        print(f"Result: {msg['content']['data']}")
                except:
                    break
            
            print("✅ Successfully connected to embedded kernel!")
            
        except Exception as e:
            print(f"❌ Failed to communicate with kernel: {e}")
        
        finally:
            client.stop_channels()
    
except ImportError:
    print("jupyter_client not available. Install with: pip install jupyter_client")
except Exception as e:
    print(f"Error: {e}")

Creating kernel client...
❌ Failed to communicate with kernel: 


In [ ]:
lasersManager